In [16]:
import MySQLdb
  
# csv file name 
filename = "contributions.csv"

myDB = MySQLdb.connect(host="seniordesign.cbvhmgr3ve3r.us-east-2.rds.amazonaws.com",port=3306,user="grlpwr",passwd="ourseniordesignproject",db="seniordesign")
myDB = myDB.cursor()

In [17]:
myDB.execute("DROP TABLE CIS120")
myDB.execute("CREATE TABLE CIS120 (question_number INT AUTO_INCREMENT PRIMARY KEY, "+
             "topic VARCHAR(500), subject VARCHAR(500), question VARCHAR(500), answer VARCHAR(500), num_times_referred_to INT, "+
             "follow_up_number INT, endorsed INT)")

0

In [18]:
# myDB.execute("INSERT INTO CIS120 (question_number, topic, question, answer, num_times_referred_to, " +
#              "follow_up_number, endorsed) VALUES (1, 'hw1', 'Q', 'A', 0, 0, 0)")

In [19]:
# initializing the titles and rows list 
fields = [] 
rows = [] 
  
# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 
  
    # get total number of rows 
#     print("Total no. of rows: %d"%(csvreader.line_num)) 
    
    # printing the field names 
#     print('Field names are:' + ', '.join(field for field in fields)) 

    #  printing first 5 rows 
#     print('\nFirst 5 rows are:\n') 

    curr_question = ""
    curr_text = ""
    curr_topic = ""
    curr_answer = ""
    curr_number = 0
    curr_follow_up = 0
    endorsed = 0
    
    # This will help us take the most recent version because it always comes first
    already_filled = 0
    for row in rows[:]: 
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number" and curr_number != col):
                if(curr_question != ""):
                    myDB.execute("INSERT INTO CIS120 (question_number, topic, question, answer, num_times_referred_to, " +
                     "follow_up_number, endorsed) VALUES (%d, %s, %s, %s, %d, %d, %d)" % (curr_number, curr_topic, 
                                curr_question, curr_answer, 0, curr_follow_up, endorsed))
                curr_number = int(col)
                curr_follow_up = 0
                curr_question = ""
                curr_answer = ""
                curr_topic = ""
                endorsed = 0
                
            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col
                
            elif(field == "Subject"): #We want to check if this is actually the question eventaully
                curr_topic = col
                
            elif(field == "Endorsed by Instructor"):
                if(col == "TRUE"):
                    endorsed = 1
                else:
                    endorsed = 0
                
            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    if(already_filled != 1):
                        curr_question = curr_text
                    alread_filled = 0
                elif(col == "updated_question"):
                    # update question entry to post and follow up numbers
                    if(already_filled != 1):
                        curr_question = curr_text
                        already_filled = 1
                        
                elif(col == "updated_i_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled != 1):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                    endorsed = 1
                elif(col == "started_off_i_answer"):
                    if(already_filled != 1):
                        curr_answer = curr_answer + "\n" + curr_text
                    alread_filled = 0
                    endorsed = 1
                    
                elif(col == "updated_s_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled != 1):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                elif(col == "started_off_s_answer"):
                    # append to answer - if it was previously endorsed and were appending should we keep it endorsed?
                    if(already_filled != 1):
                        curr_answer = curr_answer + "\n" + curr_text
                    alread_filled = 0
                    
                elif(col == "followup"):
                    # new entry for the new followup question 
                    if(curr_question != ""):
                        myDB.execute("INSERT INTO CIS120 (question_number, topic, question, answer, num_times_referred_to, " +
                         "follow_up_number, endorsed) VALUES (%d, %s, %s, %s, %d, %d, %d)"  % (curr_number, curr_topic, 
                                curr_question, curr_answer, 0, curr_follow_up, endorsed))
                        curr_question = ""
                        curr_answer = ""
                        endorsed = 0
                        
                    curr_follow_up = curr_follow_up + 1
                    curr_question = curr_text
                elif(col == "reply_to_followup"):
                    curr_answer = curr_answer + "\n" + curr_text
                                
                
        print('\n') 
    

FileNotFoundError: [Errno 2] No such file or directory: 'contributions.csv'

In [ ]:
myDB.execute("SELECT * FROM CIS120")
row = myDB.fetchone()
while row is not None:
  print(row)
  row = myDB.fetchone()